# Professional Services Retreat | GenAI Workshop

# Text2Cypher Graph Agent Demo

This notebook demonstrates how build and use a Text2Cypher graph agent.

## Imports

In [3]:
import os
import sys
sys.path.append("../../")

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.graphs import Neo4jGraph

load_dotenv()

from src.ps_genai_agents.agents.graph.text2cypher import create_text2cypher_graph_agent

(
    print(".env variables loaded!")
    if load_dotenv()
    else print("Unable to load .env variables.")
)

.env variables loaded!


## LLM Connection

The LLM we define here will be responsible for generating Cypher and summarizing the returned query results. In custom implementations you may decide to let different LLMs handle different tasks. Perhaps you can summarize with a lower-performance LLM in order to lower your application costs.

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

## Graph Connection

The LangChain `Neo4jGraph` class will be used to connect to our Aura instance. It will be used to automatically gather the graph schema and read from the database.

In [ ]:
graph = Neo4jGraph(
    url=os.environ.get("IQS_NEO4J_URI"),
    username=os.environ.get("IQS_NEO4J_USERNAME"),
    password=os.environ.get("IQS_NEO4J_PASSWORD"),
    refresh_schema=True,
)

## Example Queries

We store our queries in a yaml file. This allows us to easily store and modify our queries. 

In [ ]:
QUERY_LOCATION = "../../data/iqs/queries/queries.yml"

## Construct Agent

Now we can use the `ps-genai-agent` function to construct a Text2Cypher agent. 

In [2]:
agent = create_text2cypher_graph_agent(chat_llm=llm, neo4j_graph=graph, example_queries_location=QUERY_LOCATION)

### Architecture

Let's take a look at the architecture of our agent. You will need `Graphviz` installed to visualize the architecture with the below code. It should be identical to the below image.

In [5]:
# from IPython.display import Image
# Image(agent.get_graph().draw_png())

We see that there are four nodes in our architecture
* agent 
  * This is the input node. At this step the user's question is broken into subquestions and it's decided whether the question is in scope.
* text2cypher 
  * This node will generate a cypher query for the given question or subquestion and run it against the database. 
  * Results are gathered in the `state`.
* error 
  * This node will either explain why the agent can't answer a question or summarize any results that the agent has gathered so far.
* final_answer
  * This node will return a `Response` object containing the information gathered during the request.



There are two types of relationships in our architecture
* conditional edges
  * These edges exist where a node may have many options for the next action. The LLM decides which path shuold be taken.
* regular edges
  * These edges exist where only a single path is defined.


![text2cypher-graph-architecture](./images/text2cypher-graph-agent.png)

### Nodes

Each node outputs a dictionary with keys that map to attributes in the `state`. Only attributes that should be modified need to be provided in the `return` statement.

```Python
def text2cypher_node(data: Dict[str, Any]) -> Dict[str, Any]:

    agent_action = data.get("agent_outcome")
    intermediate_steps = list()

    tool_params = agent_action[0].tool_input

    output = execute_text2cypher(tool_params)
    intermediate_steps.append(output["intermediate_steps"][0])
    agent_outcome = (
        agent_action[1:]
        if len(agent_action) > 1
        else [
            AgentAction(
                tool="final_answer",
                tool_input="",
                log="No more actions to perform. Moving to summarization step.",
            )
        ]
    )

    return {
        "agent_outcome": agent_outcome,
        "intermediate_steps": intermediate_steps,
        "cypher": output.get("cypher"),
        "cypher_result": output.get("cypher_result"),
    }
```

### State

The state variable is passed to each node and informs which action should be taken. Different agents will likely have different states. For example the vector search agent will have an attribute for tracking node source IDs for citations. 

We annotate attributes with `operator.add` to inform `LangGraph` that new values should be appended to the list, not replace the previous values. This allows us to track results from multi-hop workflows.

```python
class AgentState(TypedDict):
    # The input string
    input: str
    # The sub questions identified in the user input
    sub_questions: Annotated[Union[List[str], None], operator.add]
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    # The cypher query ran
    cypher: Annotated[Union[List[str], None], operator.add]
    # The result of the cypher query
    cypher_result: Annotated[Union[List[Any], None], operator.add]
```

## Questions

In [4]:
res = agent.invoke(
    {
        "input": "What are the total responses under seat23 for honda civic, what is the male to female proportion for these responses and what is the problem for seat23?",
        "chat_history": [],
    }
)

> run_agent
initial data:  {'input': 'What are the total responses under seat23 for honda civic, what is the male to female proportion for these responses and what is the problem for seat23?', 'sub_questions': None, 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'cypher': None, 'cypher_result': None}
tool='Text2Cypher' tool_input={'query': 'What are the total responses under seat23 for Honda Civic?'} log="\nInvoking: `Text2Cypher` with `{'query': 'What are the total responses under seat23 for Honda Civic?'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FCbUUCsARZmGNS0ieZ21T9gB', 'function': {'arguments': '{"query": "What are the total responses under seat23 for Honda Civic?"}', 'name': 'Text2Cypher'}, 'type': 'function'}, {'id': 'call_k7aZJs6nq5jYctuTs7JwqkVc', 'function': {'arguments': '{"query": "What is the male to female proportion for responses under seat23 for Honda Civic?"}', 'name': 'Text2Cypher'}, 'type': 'funct

In [5]:
res["agent_outcome"].display()


Question:
What are the total responses under seat23 for honda civic, what is the male to female proportion for these responses and what is the problem for seat23?

Sub Questions:
What are the total responses under seat23 for Honda Civic?
What is the male to female proportion for responses under seat23 for Honda Civic?
What is the problem for seat23?

            
Cypher:
cypher
MATCH (p:Problem {id: "SEAT23"})<-[:HAS_PROBLEM]-(v:Verbatim {make: "Honda", model: "Civic"})
WITH p.problem AS problem, COUNT(v) AS totalResponses
RETURN totalResponses, problem


cypher
MATCH (p:Problem {id: "SEAT23"})<-[:HAS_PROBLEM]-(v:Verbatim {make: "Honda", model: "Civic"})
WITH p.problem AS problem, COUNT(v) AS totalResponses, 
SUM(CASE WHEN v.gender = "Male" THEN 1 ELSE 0 END) AS males,
SUM(CASE WHEN v.gender = "Female" THEN 1 ELSE 0 END) AS females
RETURN totalResponses, males, females, toFloat(males) /  (CASE WHEN females = 0 THEN 1 ELSE females END) AS maleToFemaleRatio, problem



MATCH (p:Problem 

In [6]:
res = agent.invoke({"input": "What are the top 5 problems about seats for each age buckets for men over the age of 53?", "chat_history": list()})

> run_agent
initial data:  {'input': 'What are the top 5 problems about seats for each age buckets for men over the age of 53?', 'sub_questions': None, 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'cypher': None, 'cypher_result': None}
tool='Text2Cypher' tool_input={'query': 'What are the top 5 problems about seats for each age buckets for men over the age of 53?'} log="\nInvoking: `Text2Cypher` with `{'query': 'What are the top 5 problems about seats for each age buckets for men over the age of 53?'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_m0nqHlpmbz75CIiZ7WOTdjpx', 'function': {'arguments': '{"query":"What are the top 5 problems about seats for each age buckets for men over the age of 53?"}', 'name': 'Text2Cypher'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 217, 'total_tokens': 254, 'completion_tokens_details': {'reasoning_tokens': 0}}, 

In [7]:
res.get("agent_outcome").display()


Question:
What are the top 5 problems about seats for each age buckets for men over the age of 53?

Sub Questions:
What are the top 5 problems about seats for each age buckets for men over the age of 53?

            
Cypher:

MATCH (v:Verbatim)-[:HAS_PROBLEM]->(p:Problem)
WHERE toLower(p.problem) CONTAINS 'seat' AND v.gender = "Male" AND v.minAge > 53 AND v.ageBucket IS NOT NULL
WITH v.ageBucket AS ageBucket, p.problem AS problem, COLLECT(v.verbatim) AS responses
WITH ageBucket, problem, SIZE(responses) AS total, responses
WITH *
ORDER BY ageBucket, total DESC
WITH ageBucket, COLLECT(problem) AS problems, COLLECT(total) AS totals, COLLECT(responses) AS responsesList
RETURN ageBucket, problems[..5] AS problem, totals[..5] AS total, responsesList[..5] AS responses
LIMIT 5




Cypher Result:
[{'ageBucket': '55-59', 'problem': ['SEAT04: Seats have insufficient range of adjustment', 'SEAT17: Seat - Squeaks/rattles/loose/abnormal noises', 'SEAT18: Seat - Excessively uncomfortable', 'SEAT97

In [15]:
res = agent.invoke({"input": "Please summarize the verbatims for 2023 RDX for question 010 Trunk/TG Touch-Free Sensor DTU and create categories for the problems. As an output, I want the summary, corresponding categories and their verbatims", "chat_history": list()})

> run_agent
initial data:  {'input': 'Please summarize the verbatims for 2023 RDX for question 010 Trunk/TG Touch-Free Sensor DTU and create categories for the problems. As an output, I want the summary, corresponding categories and their verbatims', 'sub_questions': None, 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'cypher': None, 'cypher_result': None}
tool='Text2Cypher' tool_input={'query': 'summarize the verbatims for 2023 RDX for question 010 Trunk/TG Touch-Free Sensor DTU and create categories for the problems. Provide the summary, corresponding categories, and their verbatims.'} log="\nInvoking: `Text2Cypher` with `{'query': 'summarize the verbatims for 2023 RDX for question 010 Trunk/TG Touch-Free Sensor DTU and create categories for the problems. Provide the summary, corresponding categories, and their verbatims.'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Oyc5hges6LhFFHYPXtuMdLKk', 'function': {'argument

In [16]:
res.get("agent_outcome").display()


Question:
Please summarize the verbatims for 2023 RDX for question 010 Trunk/TG Touch-Free Sensor DTU and create categories for the problems. As an output, I want the summary, corresponding categories and their verbatims

Sub Questions:
summarize the verbatims for 2023 RDX for question 010 Trunk/TG Touch-Free Sensor DTU and create categories for the problems. Provide the summary, corresponding categories, and their verbatims.

            
Cypher: 
MATCH (q:Question{id: 10})<-[:HAS_QUESTION]-(v:Verbatim)
  WHERE v.model='RDX'
WITH v.verbatim AS verbatim, v
MATCH (v)-[:HAS_PROBLEM]->(p:Problem)-[:HAS_CATEGORY]->(c:Category)
RETURN c.id AS category, p.problem AS problem, COLLECT(verbatim) AS verbatims
ORDER BY category



Cypher Result:
[{'category': 'Exterior', 'problem': "EXT10: Trunk/hatch/tailgate - Touch-free sensor doesn't work consistently/DTU", 'verbatims': ['when I move my foot under sensor sometimes the tailgate opens and sometimes it does not. It seems to be hit or miss.', 'f

In [17]:
res = agent.invoke({"input": "What are the top 5 problems about seats for each age buckets for men over the age of 53?", "chat_history": []})

> run_agent
initial data:  {'input': 'What are the top 5 problems about seats for each age buckets for men over the age of 53?', 'sub_questions': None, 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'cypher': None, 'cypher_result': None}
tool='Text2Cypher' tool_input={'query': 'What are the top 5 problems about seats for each age bucket for men over the age of 53?'} log="\nInvoking: `Text2Cypher` with `{'query': 'What are the top 5 problems about seats for each age bucket for men over the age of 53?'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RkczxCKLKFESlWthnKEtAF2T', 'function': {'arguments': '{"query":"What are the top 5 problems about seats for each age bucket for men over the age of 53?"}', 'name': 'Text2Cypher'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 212, 'total_tokens': 249, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'mo

Failed to write data to connection ResolvedIPv4Address(('52.44.11.98', 7687)) (ResolvedIPv4Address(('52.44.11.98', 7687)))
Failed to write data to connection IPv4Address(('p-6adb3c47-537b-0020.production-orch-0032.neo4j.io', 7687)) (ResolvedIPv4Address(('44.199.31.19', 7687)))
Failed to write data to connection ResolvedIPv4Address(('50.16.105.214', 7687)) (ResolvedIPv4Address(('50.16.105.214', 7687)))


Cypher attempt: 1
{'intermediate_steps': [{'Text2Cypher', '{\'result\': [{\'ageBucket\': \'55-59\', \'problem\': [\'SEAT04: Seats have insufficient range of adjustment\', \'INFO20: Not enough power plugs/USB ports\', \'SEAT18: Seat - Excessively uncomfortable\', \'SEAT17: Seat - Squeaks/rattles/loose/abnormal noises\', \'SEAT97: Other Seat Problem(s)\'], \'total\': [4, 4, 4, 4, 3], \'responses\': [[\'Bottom seat cushion is on the short side. The end/length of the seat only reaches about 2inches shy of my knee making it uncomfortable on longer drives\', \'I am 6 feet tall.. Not a good fit with longer legs\', \'I’m 6’3”. A couple more inches of leg room would be nice.\', \'Should have more front tilt range for me\'], [\'Come on guys?! In todays family-EVERYONE HAS A DEVICE THAT NEEDS CHARGING!!! There should be more conveniently place usb ports for every seat.\', \'Could do with usb charging port for the back seats\', \'Could use some in back seat.\', \'It would be very helpful if the RT

In [20]:
res.get("agent_outcome").display()


Question:
What are the top 5 problems about seats for each age buckets for men over the age of 53?

Sub Questions:
What are the top 5 problems about seats for each age bucket for men over the age of 53?

            
Cypher: 
MATCH (v:Verbatim)-[:HAS_PROBLEM]->(p:Problem)
WHERE toLower(v.verbatimText) CONTAINS 'seat' AND v.gender = 'Male' AND v.minAge > 53 AND v.ageBucket IS NOT NULL
WITH v.ageBucket AS ageBucket, p.problem AS problem, COLLECT(v.verbatim) AS responses
WITH ageBucket, problem, SIZE(responses) AS total, responses
WITH * ORDER BY ageBucket, total DESC
WITH ageBucket, COLLECT(problem) AS problems, COLLECT(total) AS totals, COLLECT(responses) AS responsesList
RETURN ageBucket, problems[..5] AS problem, totals[..5] AS total, responsesList[..5] AS responses
LIMIT 5



Cypher Result:
[{'ageBucket': '55-59', 'problem': ['SEAT04: Seats have insufficient range of adjustment', 'INFO20: Not enough power plugs/USB ports', 'SEAT18: Seat - Excessively uncomfortable', 'SEAT17: Seat - 